### Projection Rectification
#### Theory
Projection rectification is the process of undoing distortions to shapes, caused by projecting a 3D object down to a 2D image.
Projection rectification can be achieved in several way, the simplest way is:

- Marking 4 points in the original image, preferrably of a recognizable shape, like a square
- Specifying the new coordinate of the 4 points that we want
- Solving for the projective transformation.

Suppose points are $x_1, x_2, x_3, x_4$ and $x_1', x_2', x_3', x_4'$, then: 
$$
Hx_i = x_i' \to H\cdot [x_1 \ x_2 \ x_3 \ x_4] = [x_1' \ x_2' \ x_3' \ x_4']
$$

Problem: $H$ is 3x3, but $[x_1 \ x_2 \ x_3 \ x_4]$ and $[x_1' \ x_2' \ x_3' \ x_4']$ are 3x4. This is because $x_i$ are given in projective coordinates that is defined only up to scale.



Approach 1: Dirty fix. Flatten H as
$$
H
    = \begin{bmatrix}
        \vec h_1  \\
        \vec h_2 \\
        \vec h_3 
    \end{bmatrix}_{9 \times 1}
$$
Where $h_i$ is the $i$-th row of $H$ transposed, of dimension $3 \times 1$, and 
$x_i = (x_i, y_i, w_i)^T$
Then the equation becomes
$$
H
    = \begin{bmatrix}
        0^T & -w_i' x_i^T & y_i'x_i^T  \\
        -w_i' x
        \vec h_2 \\
        \vec h_3 
    \end{bmatrix}_{9 \times 1}
$$
